In [9]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_ablation_nystrom as file

In [10]:
# Args setup
ntrial = 50
n = 1000
ntest = 100
d = 5
noise = 0.5
features = 1000
samples = 1000
calibrate = [False, True, True]
sample_split = [False, False, True]
marginal_loss = False
retrain_hypers = False
kernel = "gaussian"
approx = "mercer"

In [11]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="128GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 1
    },
    job_cpu=1,
    walltime="12:0:0",
    job_extra_directives = ["-p medium,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

In [12]:
# Submitting jobs
futures = []
for seed in range(ntrial):
    for i in range(len(calibrate)):
            f = client.submit(file.main,seed,n,ntest,d,noise,
                              calibrate = calibrate[i],
                              sample_split = sample_split[i],
                              marginal_loss = marginal_loss,
                              retrain_hypers = retrain_hypers,
                              features = features, samples = samples,
                              approx = approx,
                              kernel = "gaussian")
            futures += [f]

In [13]:
# Check on futures
futures

[<Future: pending, key: main-0d8151033df1731109eee98e2a1b407a>,
 <Future: pending, key: main-1da99bd17a275691c64d6db7fb76a8a3>,
 <Future: pending, key: main-30105f130db7aa1f0d7069f86b027651>,
 <Future: pending, key: main-6d8bd1898b7452a85c6fd1ced9e7f6f0>,
 <Future: pending, key: main-d9d5f9427b5c68a7f8909896688acf7c>,
 <Future: pending, key: main-886a477b8068c81c76dd4b86bde2c863>,
 <Future: pending, key: main-ba52707b37bf7f1140005fd0aad1582d>,
 <Future: pending, key: main-7389dc163c0696c8e1ba843a5c75492d>,
 <Future: pending, key: main-2cd9a109845e24b3ee4dc90662b86214>,
 <Future: pending, key: main-980a9db7be85fde14ef4c4d568d65d62>,
 <Future: pending, key: main-da7c91dd9dc2a6a053ba53d17462eafe>,
 <Future: pending, key: main-f1e0b1960eb804804bcca4e75b36cfcd>,
 <Future: pending, key: main-395dba9c6e3c94e5ac0d143385b389bf>,
 <Future: pending, key: main-86f756eecac6bd22a9025241c516f64a>,
 <Future: pending, key: main-b4a0bb1cd35f4dbe425b12b1e71454a8>,
 <Future: pending, key: main-39beebf747c

In [ ]:
# Getting results
results = client.gather(futures)

In [ ]:
# Closing client
client.close()
cluster.close()

In [ ]:
# Saving results
torch.save(obj = results,
           f = "ablation_nystrom_ntrial={0}_n={1}_d={2}_noise={3}.pt".format(ntrial,n,d,noise))